In [1]:
import os
import sys 
os.chdir("/workspaces/dev/modules")
sys.path.append("/workspaces/dev/models/R-CNN")
os.environ["TF_XLA_FLAGS"] = "--tf_xla_auto_jit=2 --tf_xla_cpu_global_jit=false"
os.environ["XLA_FLAGS"] = "--xla_cpu_multi_thread_eigen=false intra_op_parallelism_threads=1 inter_op_parallelism_threads=1"

In [2]:
from tensorflow import keras
import tensorflow as tf

2025-03-11 13:52:45.432938: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-11 13:52:45.442454: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741701165.452903   13500 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741701165.455831   13500 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-11 13:52:45.467181: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
from Pills.GenerativeSequence import GenerativeSequence
from PillNet import PillNet

In [4]:
SOURCE_PATH = '/workspaces/dev/datasets/pills/data'
LABEL_PATH = '/workspaces/dev/datasets/pills/class_label.csv'
BACKGROUND_IMAGEES_PATH = '/workspaces/dev/data'

In [5]:
TRAIN_DATA_SIZE = 800000
VALIDATION_DATA_SIZE = 1000
BATCH_SIZE = 32
INPUT_SIZE = (512, 512, 3)

In [6]:
model = PillNet(BATCH_SIZE, 50, INPUT_SIZE)
model.summary()

I0000 00:00:1741701167.819946   13500 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9502 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Ti, pci bus id: 0000:01:00.0, compute capability: 8.9


Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Model: "pill_net"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ ROI (Sequential)                │ (None, 128, 128, 1)    │       138,337 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DetectionHead (Sequential)      │ (None, 8, 8, 4)        │       675,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ FeatureExtractor (Sequential)   │ (None, 128, 128, 64)   │        32,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ CentroidHead (Sequential)       │ (None, 2)              │         6,958 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SegmentationHead (Sequential)   │ (None, 128, 128, 1)    │        34,385 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ClassificationHead (Sequential) │ (None, 50)             │       210,728 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,097,932 (4.19 MB)

 Trainable params: 1,097,932 (4.19 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
train_sequence = GenerativeSequence(
  TRAIN_DATA_SIZE, BATCH_SIZE, input_size = INPUT_SIZE[:2],material_path=SOURCE_PATH,
  label_path=LABEL_PATH, background_images_path=BACKGROUND_IMAGEES_PATH,
  use_multiprocessing=True, workers = 12
)
validation_sequence = GenerativeSequence(
  VALIDATION_DATA_SIZE, BATCH_SIZE, input_size = INPUT_SIZE[:2],material_path=SOURCE_PATH,
  label_path=LABEL_PATH, background_images_path=BACKGROUND_IMAGEES_PATH,
  use_multiprocessing=True, workers = 8
)
train_dataset = tf.data.Dataset.from_generator(
  lambda: train_sequence, 
  output_signature=(
    tf.TensorSpec(shape=(BATCH_SIZE, INPUT_SIZE[0], INPUT_SIZE[1], INPUT_SIZE[2]), dtype=tf.float32),
    {
      "roi": tf.TensorSpec(shape=(BATCH_SIZE, INPUT_SIZE[0], INPUT_SIZE[1]), dtype=tf.bool),
      "centroid": tf.TensorSpec(shape=(BATCH_SIZE * 16, 2), dtype=tf.float32),
      "detection": tf.TensorSpec(shape=(BATCH_SIZE * 16, 4), dtype=tf.float32),
      "segmentation": tf.TensorSpec(shape=(BATCH_SIZE * 16, INPUT_SIZE[0]//4, INPUT_SIZE[1]//4), dtype=tf.bool),
      "classification": tf.TensorSpec(shape=(BATCH_SIZE * 16), dtype=tf.uint32)
    }
  )
)

In [8]:
model.compile(jit_compile=False)

In [9]:
checkpoint_cb = keras.callbacks.ModelCheckpoint(
  filepath="/workspaces/dev/models/R-CNN/checkpoints/pill_net.keras",
  monitor="val_loss",
  save_best_only=True,
  save_weights_only=False,
  verbose = 1
)

In [10]:
early_stopping_cb = keras.callbacks.EarlyStopping(
  monitor="val_loss",
  patience=10,
  restore_best_weights=True,
  verbose = 1
)

In [ ]:
model.fit(
  train_sequence,
  validation_data=validation_sequence,
  epochs=100,
  callbacks=[checkpoint_cb, early_stopping_cb],
  verbose=1
)

Epoch 1/100


I0000 00:00:1741701209.972623   13625 service.cc:148] XLA service 0x7f3c5c0155c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1741701209.972671   13625 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Ti, Compute Capability 8.9
I0000 00:00:1741701209.980483   13625 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1741701210.039054   13625 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2025-03-11 13:53:37.538964: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-03-11 13:53:38.906323: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng4{} for conv (f32[32,48,128,128]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,3,128,128]{3,2,1,0}, f32[48,1,9,9]{3,2,1,0}), window={size=9x9 pad=4_4x4_4}, dim_labels=bf01_

In [ ]:
model.save("/workspaces/dev/models/R-CNN/checkpoints/pill_net_3.keras")